In [1]:
import csv,random,math

def loadcsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [2]:
def splitDataset(dataset, splitRatio) :
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [80]:
def seperateByClass(dataset) :
    seperated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in seperated):
            seperated[vector[-1]] = []
        seperated[vector[-1]].append(vector)
    return seperated

In [81]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stddev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers]) / float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stddev(attribute)) for attribute in zip(*dataset)]
    
    del summaries[-1]
    return summaries

def summarizeByClass(dataset):
    seperated = seperateByClass(dataset)
    summaries = {}
    for classValue, instances in seperated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [82]:
def calculateProbablity(x, mean, stddev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stddev,2))))
    return (1 / (math.sqrt(2*math.pi) * stddev)) * exponent

def calculateClassProbablities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stddev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbablity(x, mean, stddev)
    return probabilities

In [83]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbablities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue       
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if(testSet[i][-1] == predictions[i]):
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [84]:
def NaiveBayes():
    filename = '/home/student/Fiona/datasheet/pima-indians-diabetes.csv'
    splitRatio = 0.8
    dataset = loadcsv(filename)
    
    print("\nThe length of the Data Set :",len(dataset))
    print("\nThe Data Set Splitting into Training and Testing \n")
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    
    print("\nNumber of Rows in Training Set:{0} rows".format(len(trainingSet)))
    print("\nNumber of Rows in Testing Set:{0} rows".format(len(testSet)))
    print("\nFirst Five Rows of Training Set:\n")
    for i in range(0,5):
        print(trainingSet[i],"\n")
    print("\nFirst Five Rows of Testing Set:\n")
    for i in range(0,5):
        print(testSet[i],"\n")  
    
    summaries = summarizeByClass(trainingSet)
    print("\nModel Summaries:\n",summaries)
    
    predictions = getPredictions(summaries, testSet)
    print("\nPredictions:\n",predictions)
    
    accuracy = getAccuracy(testSet, predictions)
    print("\nAccuracy: {0}%".format(accuracy))
    

In [85]:
NaiveBayes()


The length of the Data Set : 768

The Data Set Splitting into Training and Testing 


Number of Rows in Training Set:614 rows

Number of Rows in Testing Set:154 rows

First Five Rows of Training Set:

[7.0, 133.0, 88.0, 15.0, 155.0, 32.4, 0.262, 37.0, 0.0] 

[12.0, 100.0, 84.0, 33.0, 105.0, 30.0, 0.488, 46.0, 0.0] 

[1.0, 122.0, 90.0, 51.0, 220.0, 49.7, 0.325, 31.0, 1.0] 

[2.0, 93.0, 64.0, 32.0, 160.0, 38.0, 0.674, 23.0, 1.0] 

[1.0, 82.0, 64.0, 13.0, 95.0, 21.2, 0.415, 23.0, 0.0] 


First Five Rows of Testing Set:

[10.0, 139.0, 80.0, 0.0, 0.0, 27.1, 1.441, 57.0, 0.0] 

[5.0, 166.0, 72.0, 19.0, 175.0, 25.8, 0.587, 51.0, 1.0] 

[7.0, 107.0, 74.0, 0.0, 0.0, 29.6, 0.254, 31.0, 1.0] 

[9.0, 119.0, 80.0, 35.0, 0.0, 29.0, 0.263, 29.0, 1.0] 

[13.0, 145.0, 82.0, 19.0, 110.0, 22.2, 0.245, 57.0, 0.0] 


Model Summaries:
 {0.0: [(3.1903553299492384, 2.9390428546397884), (109.61928934010152, 26.238720751386683), (68.18781725888324, 18.084203777517413), (20.241116751269036, 14.89608568464373), 